# Cora Example

## Data

In [13]:
import json
import numpy as np
import torch
import scipy.sparse as sparse
from dgl.data import CoraGraphDataset
dataset = CoraGraphDataset()
graph = dataset[0]

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
Extracting file to /root/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


We can save the data file in only one file `cora.npz`.

In [14]:
# cora_feat
node_feats = sparse.csr_matrix(graph.ndata["feat"].numpy())
# cora_class
node_class = graph.ndata["label"].numpy()  # (2708,)
# cora_edge
edge = torch.stack(graph.edges()).numpy().T
# cora only has 1 single connected graph
node_list = np.ones((1, graph.num_nodes()))  # (1, 2708)
edge_list = np.ones((1, graph.num_edges()))  # (1, 10556)

data = {
    "node_feats": node_feats,
    "node_class": node_class,
    "edge": edge,
    "node_list": node_list,
    "edge_list": edge_list
}

In [15]:
np.savez_compressed("cora.npz", **data)

In [16]:
cora = np.load("cora.npz", allow_pickle=True)
cora.files

['node_feats', 'node_class', 'edge', 'node_list', 'edge_list']

In [17]:
!du cora.npz -h

116K	cora.npz


## Task

In [11]:
train_set = graph.ndata["train_mask"].nonzero().squeeze().numpy()
val_set = graph.ndata["val_mask"].nonzero().squeeze().numpy()
test_set = graph.ndata["test_mask"].nonzero().squeeze().numpy()

In [ ]:
task_data = {
    "train": train_set,
    "val": val_set,
    "test": test_set
}
np.savez_compressed("cora_task.npz", **task_data)